# Part 15

# Using Code Interpreter

Universal code for the entire notebook

In [ ]:
# Uncomment the line below to make sure you have all the packages needed
# %pip install -r requirements.txt

In [ ]:
# Import necessary libraries
from openai import OpenAI  # Used for interacting with OpenAI's API
from typing_extensions import override  # Used for overriding methods in subclasses
from openai import AssistantEventHandler  # Used for handling events related to OpenAI assistants

In [ ]:
# Create an instance of the OpenAI class to interact with the API.
# This assumes you have set the OPENAI_API_KEY environment variable.
client = OpenAI() 

In [ ]:
# Event handler class to handle events related to streaming output from the assistant
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nASSISTANT MESSAGE >\n", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nASSISTANT MESSAGE >\n{tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))

## Creating an Assistant with Code Interpreter Enabled

Our first step is to create an Assistant that can use Code Interpreter